In [1]:

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ['TF_KERAS'] = '1'

import tensorflow as tf
import keras_bert
from functools import partial
import importlib
import data_loader
importlib.reload(data_loader)

DataLoader = data_loader.DataLoader

# keras_bert.backend.TF_KERAS = 1

config = './config/hub2-1.json'
print(keras_bert.backend.TF_KERAS)

1


In [2]:
dl = DataLoader(config=config, data='../data', n_max=None)
config_file = os.path.join(dl.data, 'bert_model', 'bert_config.json')
checkpoint_file = os.path.join(dl.data, 'bert_model', 'bert_model.ckpt')

INFO:root:DataLoader initializing
INFO:root:reading lab data from ../data/soundAttGAN & ../data/soundAttGAN/koreancorpus.xlsx
INFO:root:total number of data: 1925
INFO:root:reading hub data from ../data/KsponSpeech_01
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0094
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0090
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0048
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0016
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0020
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0074
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0093
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0001
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0105
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0052
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0017
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0039
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0076
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0031
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0087
DEBUG:root:../dat

In [3]:
it = iter(dl.train_generator(data='hub', return_text=True))

In [4]:
next(it)

[101,
 8924,
 31401,
 25503,
 118671,
 9682,
 9539,
 18784,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [5]:
def _get_embedding_table():
    ckpt_loader = tf.train.load_checkpoint(checkpoint_file)
#     model = keras_bert.load_trained_model_from_checkpoint(config_file=config_file,
#                                                          checkpoint_file=checkpoint_file,
#                                                          training=False,
#                                                          trainable=None,
#                                                          output_layer_num=1,
#                                                          seq_len=dl.max_len)
#     embed_table = keras_bert.get_token_embedding(model)
#     del(model)
    embed_table = ckpt_loader.get_tensor('bert/embeddings/word_embeddings')
    return embed_table

In [22]:
import tensorflow as tf
from utils import *

keras = tf.keras

BatchNorm = keras.layers.BatchNormalization
Conv2D = keras.layers.Conv2D
DConv = keras.layers.Conv2DTranspose
Dense = keras.layers.Dense


class AutoEncoder(keras.models.Model):
    def __init__(self, config, input_shape, **kwargs):
        super(AutoEncoder, self).__init__(**kwargs)
        self.config = load_config(config)
        self.audio_shape = input_shape

        self.act_fn = keras.layers.LeakyReLU()
        self.kernel_size = self.config['kernel_size']
        
        self.embedding_table = _get_embedding_table()

        self.conv = [
            Conv2D(filters=32, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None,
                   input_shape=input_shape),
            BatchNorm(),
            self.act_fn,
            Conv2D(filters=64, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            Conv2D(filters=128, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            Conv2D(filters=256, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            Conv2D(filters=512, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            #             keras.layers.Flatten(),
            #             Dense(units=output_size, activation=None),
            #             self.act_fn
        ]

        self.dconv = [
            # 16
            DConv(filters=512, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            # 32
            DConv(filters=256, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,
            # 64
            DConv(filters=128, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,

            DConv(filters=64, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            BatchNorm(),
            self.act_fn,

            DConv(filters=1, kernel_size=self.kernel_size, strides=(2, 2), padding='same', activation=None),
            keras.layers.ReLU(max_value=80.0)
            #             keras.layers.Activation('sigmoid')
            # 128
        ]
#         self.embed_hidden = Dense(units=self.hidden_size[0] * self.hidden_size[1])

    @tf.function
    def call(self, inputs, **kwargs):
        audio, text = inputs
        audio = tf.expand_dims(audio, -1)
#         text = tf.gather(self.embedding_table, text)
        # text embed size: 768

        for layer in self.conv:
            #             print(inputs.shape)
            audio = layer(audio)
        for layer in self.dconv:
            #             print(inputs.shape)
            audio = layer(audio)
        audio = tf.squeeze(audio, axis=-1)
        return audio
    
    @property
    def hidden_size(self):
        return int(self.audio_shape[0] / 2**5), int(self.audio_shape[1] / 2**5)
        
        


In [23]:
autoencoder = AutoEncoder(config=config, input_shape=dl.stft_shape)

In [18]:
autoencoder.hidden_size[0]

8

In [19]:
autoencoder.trainable_variables

[]

In [20]:
dataset_audio = tf.data.Dataset.from_generator(partial(dl.train_generator, data='hub'), output_types=tf.float32)
dataset_text = tf.data.Dataset.from_generator(partial(dl.train_generator, data='hub', return_text=True), output_types=tf.int32)

dataset = tf.data.Dataset.zip((dataset_audio, dataset_text))
dataset = dataset.shuffle(buffer_size=dl.batch_size * 10).batch(16)

In [14]:
inputs = next(iter(dataset))

In [21]:
autoencoder(inputs)

    @tf.function
    def call(self, inputs, **kwargs):
        audio, text = inputs
        audio = tf.expand_dims(audio, -1)
#         text = tf.gather(self.embedding_table, text)
        # text embed size: 768

        for layer in self.conv:
            #             print(inputs.shape)
            audio = layer(audio)
        for layer in self.dconv:
            #             print(inputs.shape)
            audio = layer(audio)
        audio = tf.squeeze(audio, axis=-1)
        return audio

This may be caused by multiline strings or comments not indented at the same level as the code.


    @tf.function
    def call(self, inputs, **kwargs):
        audio, text = inputs
        audio = tf.expand_dims(audio, -1)
#         text = tf.gather(self.embedding_table, text)
        # text embed size: 768

        for layer in self.conv:
            #             print(inputs.shape)
            audio = layer(audio)
        for layer in self.dconv:
            #             print(inputs.shape)
            audio = layer(audio)
        audio = tf.squeeze(audio, axis=-1)
        return audio

This may be caused by multiline strings or comments not indented at the same level as the code.


    @tf.function
    def call(self, inputs, **kwargs):
        audio, text = inputs
        audio = tf.expand_dims(audio, -1)
#         text = tf.gather(self.embedding_table, text)
        # text embed size: 768

        for layer in self.conv:
            #             print(inputs.shape)
            audio = layer(audio)
        for layer in self.dconv:
            #             print(inputs.shape)
            audio = layer(audio)
        audio = tf.squeeze(audio, axis=-1)
        return audio

This may be caused by multiline strings or comments not indented at the same level as the code.


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_5/Conv2D (defined at /home/mhlee/miniconda3/envs/jupyter/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_call_3090]

Function call stack:
call


In [23]:
print(checkpoint_file)

../data/bert_model/bert_model.ckpt


In [27]:
dl.data + '/bert_model'

'../data/bert_model'

In [38]:
checkpoint = tf.train.Checkpoint()

In [41]:

ckpt_manager = tf.train.CheckpointManager(checkpoint, dl.data + '/bert_model', max_to_keep=10)

In [45]:
ckpt_manager.latest_checkpoint

In [53]:
for key, value in ckpt_loader.get_variable_to_shape_map().items():
    if 'embeddings' in key:
        print(key, ':', value)
    # bert/embeddings/word_embeddings 사용

bert/embeddings/token_type_embeddings : [2, 768]
bert/embeddings/word_embeddings : [119547, 768]
bert/embeddings/LayerNorm/beta : [768]
bert/embeddings/position_embeddings : [512, 768]
bert/embeddings/LayerNorm/gamma : [768]


In [56]:
embed_table = tf.Variable(tf.zeros([119547, 768]), name='bert/embeddings/word_embeddings')

In [ ]:
embed_table

In [58]:
embed_table = ckpt_loader.get_tensor('bert/embeddings/word_embeddings')

In [59]:
embed_table

array([[ 0.02595074, -0.00617341, -0.00409975, ...,  0.02965234,
         0.02417551,  0.01970279],
       [ 0.01038065, -0.0136286 ,  0.00672081, ...,  0.01237162,
         0.0267217 ,  0.03370738],
       [ 0.0220679 , -0.00360613,  0.01932366, ...,  0.0069061 ,
         0.026809  ,  0.00498276],
       ...,
       [ 0.00684139,  0.01885802,  0.02666426, ...,  0.02292391,
         0.06465269,  0.04373793],
       [ 0.0183579 ,  0.01480132,  0.02434449, ...,  0.03205629,
         0.00708906,  0.02039703],
       [ 0.02139908,  0.01879423, -0.01343376, ..., -0.00597953,
         0.00583893, -0.00586251]], dtype=float32)